# Memory Guides

docs: https://python.langchain.com/en/latest/modules/memory/how_to_guides.html

### Memory Types
- ConversationBufferMemory
- ConversationBufferWindowMemory
- Entity Memory
- Conversation Knowledage Graph Memory
- ConversationSummaryMemory
- ConversationSummaryBufferMemory
- ConversationTokenBufferMemory
- VectorStore-Backed Memory

### 重点

这些Memory, 构造参数如果添加`return_messages=True` 则一般与聊天模型一起使用

### ConversationBufferMemory

docs: https://python.langchain.com/en/latest/modules/memory/types/buffer.html

In [1]:
from app.init import init_llm
init_llm()

from langchain.memory import ConversationBufferMemory

Init OpenAI env


In [4]:
# memory = ConversationBufferMemory()

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({'input': 'hi'}, {'output': 'whats up'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}, example=False),
  AIMessage(content='whats up', additional_kwargs={}, example=False)]}

In [5]:
# using this in a chain (setting verbose=True  will see the prompt)
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

chain = ConversationChain(llm = OpenAI(), memory = ConversationBufferMemory(), verbose=True)
chain.predict(input='你知道三国时期有哪几个国家，它们的国君分别是谁吗？')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你知道三国时期有哪几个国家，它们的国君分别是谁吗？
AI:

> Finished chain.


' 是的，我知道三国时期有曹魏，蜀汉，吴国和东吴四个国家，曹魏的国君是曹操，蜀汉的国君是刘备，吴国的国君是孙权，东吴的国君是孙策。'

In [6]:
chain.predict(input='当时哪个国家最强大?')



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你知道三国时期有哪几个国家，它们的国君分别是谁吗？
AI:  是的，我知道三国时期有曹魏，蜀汉，吴国和东吴四个国家，曹魏的国君是曹操，蜀汉的国君是刘备，吴国的国君是孙权，东吴的国君是孙策。
Human: 当时哪个国家最强大?
AI:

> Finished chain.


' 在三国时期，曹魏是最强大的国家，但东吴也有强大的实力。'

---

### ConversationBufferWindowMemory
docs: https://python.langchain.com/en/latest/modules/memory/types/buffer_window.html

**相当于滑动窗口**

>ConversationBufferWindowMemory 会保留一份对话随时间变化的交互列表。它只使用最后 K 个交互。这对于保持最近交互的滑动窗口很有用，因此缓冲区不会变得太大

In [7]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2) # 最近的2次对话

memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
memory.save_context({"input": "great"}, {"output": "thanks"})

memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much\nHuman: great\nAI: thanks'}

In [8]:
# using in a chain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

conversation_with_window = ConversationChain(
    llm=OpenAI(temperature=0), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_window.predict(input="二战期间，哪个国家牺牲最多？")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 二战期间，哪个国家牺牲最多？
AI:

> Finished chain.


' 根据历史记载，二战期间，苏联牺牲最多，其次是中国，紧随其后的是日本。苏联在二战期间牺牲了约2700万人，中国牺牲了约1400万人，日本牺牲了约800万人。'

In [9]:
conversation_with_window.predict(input="它们分别做出了什么贡献")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 二战期间，哪个国家牺牲最多？
AI:  根据历史记载，二战期间，苏联牺牲最多，其次是中国，紧随其后的是日本。苏联在二战期间牺牲了约2700万人，中国牺牲了约1400万人，日本牺牲了约800万人。
Human: 它们分别做出了什么贡献
AI:

> Finished chain.


' 苏联在二战期间为自由世界作出了巨大贡献，其中包括抗击德国军队，抵抗纳粹的压迫，以及在战争中取得的胜利。中国也发挥了重要作用，其中包括抗击日本军队，抵抗日本的侵略，以及在战争中取得的胜利。日本也发挥了重要作用，其中包括抗击苏联军队，抵抗苏'

In [10]:
conversation_with_window.predict(input="它们之中当时哪个国家最强大?")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 二战期间，哪个国家牺牲最多？
AI:  根据历史记载，二战期间，苏联牺牲最多，其次是中国，紧随其后的是日本。苏联在二战期间牺牲了约2700万人，中国牺牲了约1400万人，日本牺牲了约800万人。
Human: 它们分别做出了什么贡献
AI:  苏联在二战期间为自由世界作出了巨大贡献，其中包括抗击德国军队，抵抗纳粹的压迫，以及在战争中取得的胜利。中国也发挥了重要作用，其中包括抗击日本军队，抵抗日本的侵略，以及在战争中取得的胜利。日本也发挥了重要作用，其中包括抗击苏联军队，抵抗苏
Human: 它们之中当时哪个国家最强大?
AI:

> Finished chain.


' 根据历史记载，二战期间，苏联是最强大的国家，其次是美国，紧随其后的是中国和日本。苏联在二战期间拥有最大的军队，最先进的武器，以及最强大的经济实力。'

In [11]:
conversation_with_window.predict(input="它们之中哪些是法西斯")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 它们分别做出了什么贡献
AI:  苏联在二战期间为自由世界作出了巨大贡献，其中包括抗击德国军队，抵抗纳粹的压迫，以及在战争中取得的胜利。中国也发挥了重要作用，其中包括抗击日本军队，抵抗日本的侵略，以及在战争中取得的胜利。日本也发挥了重要作用，其中包括抗击苏联军队，抵抗苏
Human: 它们之中当时哪个国家最强大?
AI:  根据历史记载，二战期间，苏联是最强大的国家，其次是美国，紧随其后的是中国和日本。苏联在二战期间拥有最大的军队，最先进的武器，以及最强大的经济实力。
Human: 它们之中哪些是法西斯
AI:

> Finished chain.


' 根据历史记载，二战期间，德国、日本和意大利是法西斯国家。这三个国家都支持纳粹的政策，并且在战争中支持纳粹的侵略。'

---
### ConversationEntityMemory
docs: https://python.langchain.com/en/latest/modules/memory/types/entity_summary_memory.html

记住关于特定实体的事情。它提取实体上的信息(使用LLMs)，随着时间的推移构建其关于该实体的知识(也使用LLMs)。

In [22]:
from langchain.llms import OpenAI
from langchain.memory import ConversationEntityMemory
llm = OpenAI(temperature=0)

memory = ConversationEntityMemory(llm=llm)

_input = {"input": "Deven & Sam are working on a hackathon project"}
memory.load_memory_variables(_input)

memory.save_context(
    _input,
    {"output": " That sounds like a great project! What kind of project are they working on?"}
)

memory.load_memory_variables({"inputs": "who is Sam?"})

{'history': 'Human: Deven & Sam are working on a hackathon project\nAI:  That sounds like a great project! What kind of project are they working on?',
 'entities': {'Sam': 'Sam is working on a hackathon project with Deven.'}}